In [19]:
import pandas as pd
import numpy as np

star_df = pd.read_csv('celeb_name_list.csv')        # 연예인 리스트
article_file = open('data.txt', 'r')      # 크롤링한 기사 제목들의 텍스트 파일 (구분자는 '\n'으로 가정)

# articles_dict: {name: <연예인 이름 리스트>, article: <기사 제목 리스트>}
articles_dict = {'Name':[], 'Article':[]}

star_df.head(10)

,ID,Name,Category,Age
0,2470,BMK,가수,47
1,2469,JK김동욱,가수,44
2,2468,KAI,가수,26
3,2467,MC딩동,배우,41
4,2466,MC몽,가수,41
5,2465,RM,가수,26
6,2464,T.O.P,가수,32
7,2463,가인,가수,33
8,2462,가희,배우,39
9,2461,간미연,가수,38


# articles_dict 생성 후 데이터프레임 만들기
: 기사 제목에 특정 연예인 이름이 들어있으면 'Article', 'Name' 키에 각각 저장 후 article_df 생성

* article: 기사 제목
* star_df: celeb_name_list.csv의 데이터프레임. star_df['Name']에 연예인 이름(=name)이 저장되어있음
* article_df: articles_dict의 데이터프레임. columns=['Name', 'Article'] 


In [20]:
while True:
    # 크롤링한 기사 제목 한 줄을 읽어옴
    article = article_file.readline()
    # 크롤링한 기사 제목을 모두 읽었으면 break
    if article == '':
        break
    #article = article[:-1]       # 기사 제목 뒤의 \n 제외
    # namelist 안의 모든 연예인 이름에 대해서 기사 제목에 포함되는 이름이 있는 지 확인
    for name in star_df['Name']:
        # 보통 연예인 이름 뒤에 컴마가 오길래 '<이름>,'를 찾는걸로 해봄
        if name+',' in article or name+' ' in article or name+"'" in article:
            # 이름으로 쓰인 단어가 아닐 때는 제외함 
            if article.find(name) != 0 and article[article.find(name)-1] != ' ':
                continue
            
            if not (name in articles_dict['Name']):
                articles_dict['Name'].append(name)
                articles_dict['Article'].append(article)
            else:
                index = articles_dict['Name'].index(name)
                if not (article in articles_dict['Article'][index]):
                    articles_dict['Article'][index] += article
                
        # 제목에 포함된 연예인을 찾았어도 연예인 리스트 전체 탐색함
        # => 기사에 두 명 이상의 연예인이 있을 경우 모두 찾음
article_df = pd.DataFrame(articles_dict)

In [21]:
article_df.head(10)

,Name,Article
0,가희,"가희, 등산하며 꾸준한 자기관리..건강미 넘치는 미소[★SHOT!]\tOSEN\n가..."
1,김혜수,"김혜수, 섹시 건강미의 대명사…마스크 써도 우아함 철철\t헤럴드POP\n김혜수, 건..."
2,김성령,"김성령, 건강미 넘치는 프리다이빙…""54세 몸매 맞아?""\t머니투데이\n김성령 “물..."
3,이하늬,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\t머니투데이\n이하늬, 독..."
4,김희정,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\t헤럴드POP\n김희정, 건강미 넘..."
5,류시원,"류시원, 재혼 후 근황…49세 나이 믿기지 않는 동안+건강미\t한국경제\n"
6,최여진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\t뉴스엔\n최여진, ..."
7,류화영,"류화영, 건강미 돋보이는 테니스장에서의 하루\t뉴스컬처\n류화영 ""#건강미 #섹시한..."
8,유이,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\t헤럴드POP\..."
9,신현빈,"신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보' [화보]\tTV리포트\n"


# count_dict 생성 후 데이터프레임 만들기
: 각각의 연예인 이름이 몇 번 언급되어있는지를 저장함
'Name'키에 연예인 이름, 'Count'키에 언급 횟수를 저장 후 count_df 생성


In [22]:
# count_dict: {Name : 한 번 이상 언급된 연예인 이름 리스트, Count : 언급 횟수}
count_dict = {'Name': articles_dict['Name'], 'Count': []}
i = 0
for name in articles_dict['Name']:
    count_dict['Count'].append(articles_dict['Article'][i].count('\n'))
    i +=1

count_df = pd.DataFrame(count_dict)
count_df.head()

,Name,Count
0,가희,17
1,김혜수,4
2,김성령,2
3,이하늬,81
4,김희정,31


# 데이터프레임 전처리

1. 연예인 리스트 데이터프레임(star_df)과 연예인별 언급횟수 데이터프레임(count_df) 합침
2. 언급 되지 않은 연예인이 있는 행은 삭제 (count 컬럼이 NaN인 경우 해당 행 삭제)
3. count 컬럼을 기준으로 내림차순 정렬

In [23]:
# df에 언급 횟수인 Count 컬럼 추가
merged_df = pd.merge(star_df, count_df, on='Name', how = 'outer')
merged_df.head()

,ID,Name,Category,Age,Count
0,2470,BMK,가수,47,NaN
1,2469,JK김동욱,가수,44,NaN
2,2468,KAI,가수,26,NaN
3,2467,MC딩동,배우,41,NaN
4,2466,MC몽,가수,41,NaN


In [24]:
# 언급 되지 않은 연예인이 있는 행은 삭제
temp_df = merged_df.dropna(axis = 0, how = 'any')
# 언급 횟수 기준 내림차순으로 정렬
sorted_df = temp_df.sort_values(by = 'Count', ascending=False,)

# final_df columns: 연예인 정보 + 언급 횟수 + 언급된 기사 제목
final_df = pd.merge(sorted_df, article_df, on='Name', how = 'outer')
final_df.head(20)

,ID,Name,Category,Age,Count,Article
0,746,이하늬,배우,37,81.0,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\t머니투데이\n이하늬, 독..."
1,6,효린,가수,29,63.0,"효린, 'SAY MY NAME' 첫 뮤비티저 공개 예고…'건강미 폭발' 퍼포먼스 기..."
2,1477,설현,가수,25,54.0,"AOA 설현, 농심 '켈로그' 모델 발탁…""건강미 대표 이미지 부합""\tRPM9\n..."
3,1487,선미,가수,28,53.0,"선미 '보라빛 밤' 티저 공개, 스윔수트 입고 뽐낸 건강미\t한국경제\n‘8kg 증..."
4,1460,소유,가수,28,52.0,"소유, 신곡 '가라고' 안무 영상 공개…탄탄한 건강미\t뉴스1\n소유, 나이 29살..."
5,882,이시영,배우,38,49.0,"[오마이포토] 'SF8' 이시영, 건강미 국가대표\t오마이뉴스\n이시영, 흑백 뚫고..."
6,1127,유이,배우,32,46.0,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\t헤럴드POP\..."
7,431,조여정,배우,39,36.0,"조여정, 촉촉한 피부로 건강미 발산···고혹적인 미모 '시선 강탈'\t텐아시아\n조..."
8,209,클라라,배우,35,32.0,"클라라, 밀착 브라톱+레깅스 매치…독보적인 건강미\t머니투데이\n클라라, 운동으로 ..."
9,1922,김희정,배우,28,31.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\t헤럴드POP\n김희정, 건강미 넘..."


# 데이터 확인하기

1. 동명이인인 데이터 확인
2. 상위 20명의 후보 연예인 리스트 확인
3. 후보 연예인들이 언급되었던 기사가 제대로 된 것인지 확인
4. 최종 데이터프레임을 csv 파일로 저장

In [25]:
### 생략 가능 ###
# 동명이인일 시 두 명 다 같은 언급횟수를 가지고 있음  
# ex)김희정(49세), 김희정(28세): 기사 하나에 언급되면 둘 다 count = 1이 됨
# duplicated: 동명이인인 이름의 리스트
duplicated = final_df[final_df.Name.duplicated()]['Name'][:20].tolist()
print('=== 동명이인인 이름 리스트 ===')
print(duplicated)


# 동명이인이 들어간 기사 체크하기
for key in duplicated:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

# 동명이인의 정보 확인하기
duplicated_df = final_df[final_df['Name'].isin(duplicated)]
duplicated_df.head(100)

=== 동명이인인 이름 리스트 ===
['김희정', '한혜진', '지성', '현아', '김종국', '미나', '박준형', '은정', '김민정', '정유미', '김규리', '고은아', '최성원', '김지훈', '김지훈', '임수정']
>> 김희정
김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄	헤럴드POP
김희정, 건강미 넘치는 화보 ‘치명적인 매력’	스포츠동아
김희정, 군살 제로 건강미..튼튼 꿀벅지 '깜짝'[★SHOT!]	OSEN
김희정, 독보적 건강미 발산…다이어트도 힙하게	스포츠서울
김희정, 건강미에 더한 몽환적 분위기…치명적인 매력 [화보]	엑스포츠뉴스
김희정, 다이어트 CF로 건강미 과시	한스경제
김희정, 구릿빛 피부에 빛나는 몸매…건강미 넘치네	마이데일리
김희정, 건강미 넘치는 '워너비 몸매' 담은 티저 영상	일간스포츠
김희정, 화보 비하인드컷 공개...'러블리+건강미'	TV리포트
김희정, 구릿빛 피부+탄탄 몸매로 '섹시 건강미' 뿜뿜	헤럴드POP
김희정, 비키니·트레이닝복까지 완벽 소화…건강미 넘치는 매력 [화보]	엑스포츠뉴스
김희정, '건강미+몽환적 분위기' 발산하는 유니크한 매력 [화보]	TV리포트
김희정, 준PO 4차전 시구..복근 드러낸 '건강미'	뉴스1
배우 김희정, SNS에 건강미 넘치는 수영복 차림 공개	스포츠경향
김희정, 건강미+몽환적 분위기 ‘치명적 매력 완성’[화보]	뉴스엔
김희정, '힙'한 다이어트 티저 영상 공개…건강미 넘치는 '워너비 몸매'	엑스포츠뉴스
김희정, 건강미 넘치는 구릿빛 피부	뉴스1
[포토] 김희정, '건강미 넘치는 그녀~'	한국경제
[포토] 김희정, '건강미 넘치는 모습에 눈길~'	한국경제
[포토] 김희정, 스포티한 브라톱 래시가드로 건강미 뽐내	이데일리
김희정, 맥주 광고 모델 발탁…‘건강미 어필’	MBN
섹시 아이콘 김희정, 건강미 넘치는 래쉬가드 수영복 화보 공개	한국경제TV
김희정, 건강미 넘치는 '워너비 몸매'…다이어트도 힙하게	TV리포트
김희정, 구릿빛 피부에 돋보이는 건강미…감탄 자아내는 몸매	마이

,ID,Name,Category,Age,Count,Article
9,1922,김희정,배우,28,31.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\t헤럴드POP\n김희정, 건강미 넘..."
10,1923,김희정,배우,49,31.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\t헤럴드POP\n김희정, 건강미 넘..."
16,109,한혜진,모델,37,16.0,"[포토] 한혜진, 건강미 넘치는 미모\t스포츠조선\n'나 혼자 산다' 한혜진, '섹..."
17,110,한혜진,배우,38,16.0,"[포토] 한혜진, 건강미 넘치는 미모\t스포츠조선\n'나 혼자 산다' 한혜진, '섹..."
19,376,지성,배우,43,15.0,걸그룹 소녀주의보 지성 “60kg아이돌 안녕”···건강미 넘치는 근황\t스포츠경향\...
20,375,지성,가수,18,15.0,걸그룹 소녀주의보 지성 “60kg아이돌 안녕”···건강미 넘치는 근황\t스포츠경향\...
25,75,현아,가수,33,15.0,"‘원더풀’ 현아, 美친 건강미 [화보]\t매일경제\n현아, 건강미 넘치는 스포티+섹..."
26,74,현아,가수,28,15.0,"‘원더풀’ 현아, 美친 건강미 [화보]\t매일경제\n현아, 건강미 넘치는 스포티+섹..."
27,2038,김종국,개그맨,57,13.0,"[포토] 김종국, 한결같은 건강미\tMBC연예\n[포토] 김종국, 넘치는 건강미\t..."
28,2039,김종국,가수,44,13.0,"[포토] 김종국, 한결같은 건강미\tMBC연예\n[포토] 김종국, 넘치는 건강미\t..."


In [26]:
candidates = final_df['Name'].tolist()[:20]
print('=== Top 20 ===')
print(candidates)

=== Top 20 ===
['이하늬', '효린', '설현', '선미', '소유', '이시영', '유이', '조여정', '클라라', '김희정', '김희정', '유리', '이효리', '제시', '루나', '가희', '한혜진', '한혜진', '전혜빈', '지성']


In [27]:
# 상위 20명의 기사가 제대로 된 것인지 체크하기
print('=== Check articles ===')
for key in candidates:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

=== Check articles ===
>> 이하늬
이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미	머니투데이
이하늬, 독보적인 건강미 [화보]	스포츠동아
이하늬, 싱그러운 건강미 물씬…민낯도 자신있게	조이뉴스24
이하늬 셀카 사진 속 건강미 넘친 미소…"영국 아닌 한국 지방 촬영 중"	데일리한국
이하늬, 독보적 건강미..청바지만 입어도 ‘섹시’	MK스포츠
이하늬, 롱패딩 입고도 섹시..건강미 뽐내	이데일리
이하늬, 내츄럴한 의상·독보적인 건강미…교토 화보	국제신문
이하늬 화보, 청바지만 입어도 섹시 “독보적 건강미”	서울신문
이하늬 교토 화보, 건강미 넘치는 섹시함[포토엔]	뉴스엔
배우 이하늬, 건강미 넘치는 화보 공개	세계일보
배우 이하늬 독보적인 건강미의 교토 화보 공개!	국방일보
[화보] 이하늬, 래쉬가드 입고 건강미 발산 '상큼'	뉴스1
이하늬, 래시가드로 ‘건강미 뽐내’… 특유의 미소까지	동아일보
[SW포토] 이하늬, 독보적 건강미	스포츠월드
이하늬, 교토 화보 공개…독보적 건강미	스페셜경제
[포토] 이하늬, 건강미 넘치는 비키니 자태 ‘시선 강탈’	서울신문
이하늬, 건강미 넘치는 수영복 자태 '눈길'	월요신문
이하늬, `건강미 넘쳐~` [MBN포토]	MBN
이하늬, 여름 해변에서…건강미 넘치는 수영복 화보	매일경제
[화보] 이하늬, 건강미 넘치는 래쉬가드 자태	스포츠경향
[스타패션] 이하늬, 대체불가 건강미…해변의 여신으로 변신	스포츠경향
이하늬, 건강미 넘치는 래시가드 화보 '시선강탈'	스포츠서울
이하늬, 롱패딩으로도 가릴 수 없는 독보적인 건강미 [화보]	스포츠동아
이하늬, 비치웨어 모델 발탁 '건강미 물씬' [화보]	엑스포츠뉴스
배우 이하늬 화보 공개… 건강미 넘치는 섹시함	서울신문
[포토엔HD] 이하늬 ‘건강미 넘치는 보디라인’	뉴스엔
[OBS포토] 이하늬, 건강미 넘치는 매력	OBS
명동 들썩이게 만든 이하늬 ‘국보급 건강미’ 자태	스포츠경향
이하늬, 건강미 넘치는 완벽 바디라인[화보]	헤럴드POP
이하늬, 미스코리아의 건강미 

In [28]:
import qgrid
qgrid_widget = qgrid.show_grid(final_df,show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
# csv로 저장
final_df.to_csv('candidates_final.csv', index=False, mode='w', encoding='utf-8-sig')